In [1]:
import numpy as np
import json

In [6]:
class Battery:
    def __init__(self, config_path="battery_config.json", timestep_hours=1.0):

        with open(f"src/config/{config_path}", "r") as f:
            cfg = json.load(f)

        self.dt = timestep_hours
        self.efficiency = cfg["efficiency"]

        # -------- Randomized physical parameters --------
        self.capacity_kwh = self._sample(cfg["capacity_kwh"])
        self.max_charge_kw = self._sample(cfg["max_charge_power_kw"])
        self.max_discharge_kw = self._sample(cfg["max_discharge_power_kw"])

        # -------- Initial State of Charge --------
        soc_low, soc_high = cfg["initial_soc_range"]
        self.soc = np.random.uniform(soc_low, soc_high)

    def _sample(self, dist):
        val = np.random.normal(dist["mean"], dist["std"])
        return float(np.clip(val, dist["min"], dist["max"]))

    def charge(self, requested_energy_kwh):
        """
        Attempt to store energy into battery.
        Returns actual energy accepted (kWh).
        """

        # convert power limit to energy limit
        max_possible = self.max_charge_kw * self.dt
        energy = min(requested_energy_kwh, max_possible)

        # available space
        available_space = (1 - self.soc) * self.capacity_kwh
        energy = min(energy, available_space)

        # apply efficiency (loss occurs during charging)
        stored_energy = energy * self.efficiency

        self.soc += stored_energy / self.capacity_kwh
        self.soc = min(self.soc, 1.0)

        return energy  # energy drawn from grid/PV

    # --------------------------------------------------
    # Discharging
    # --------------------------------------------------

    def discharge(self, requested_energy_kwh):
        """
        Attempt to supply energy from battery.
        Returns actual energy delivered (kWh).
        """

        max_possible = self.max_discharge_kw * self.dt
        energy = min(requested_energy_kwh, max_possible)

        available_energy = self.soc * self.capacity_kwh
        energy = min(energy, available_energy)

        # inverter loss during discharge
        delivered_energy = energy * self.efficiency

        self.soc -= energy / self.capacity_kwh
        self.soc = max(self.soc, 0.0)

        return delivered_energy

    # --------------------------------------------------
    # Utility
    # --------------------------------------------------

    def energy_stored(self):
        return self.soc * self.capacity_kwh

    def reset(self):
        # randomize initial SOC at new episode
        self.soc = np.random.uniform(0.3, 0.8)

In [7]:
battery=Battery()

In [34]:
battery.charge(1)

0.6778129631738448

In [38]:
battery.energy_stored()

7.063587926119938